## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-22-14-55-11 +0000,nypost,The eight airports named for presidents as Tru...,https://nypost.com/2026/02/22/lifestyle/the-ei...
1,2026-02-22-14-54-27 +0000,nyt,U.S.-Canada Tensions Turn Olympic Hockey Final...,https://www.nytimes.com/2026/02/22/world/europ...
2,2026-02-22-14-53-01 +0000,nyt,Denmark Rejects Trump’s Plan to Send Hospital ...,https://www.nytimes.com/2026/02/22/us/politics...
3,2026-02-22-14-45-25 +0000,nyt,"Man Is Fatally Shot at Mar-a-Lago, Secret Serv...",https://www.nytimes.com/2026/02/22/us/politics...
4,2026-02-22-14-45-01 +0000,startribune,Armed man shot and killed after entering secur...,https://www.startribune.com/us-secret-service-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2423/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
4,trump,43
657,tariff,15
410,new,14
127,tariffs,12
164,global,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
179,2026-02-21-16-56-26 +0000,cbc,Trump says he's raising new global tariff rate...,https://www.cbc.ca/news/politics/trump-global-...,116
171,2026-02-21-17-23-00 +0000,wsj,President Trump said he would increase to 15% ...,https://www.wsj.com/politics/policy/trump-incr...,113
181,2026-02-21-16-46-07 +0000,wapo,Trump’s response to Supreme Court tariff rulin...,https://www.washingtonpost.com/business/2026/0...,104
78,2026-02-22-06-32-45 +0000,nypost,Trump reveals his ‘new hero’ Supreme Court jus...,https://nypost.com/2026/02/22/us-news/trump-ca...,99
72,2026-02-22-08-08-24 +0000,bbc,Trump says he will increase his new global tar...,https://www.bbc.com/news/articles/cn8z48xwqn3o...,96


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
179,116,2026-02-21-16-56-26 +0000,cbc,Trump says he's raising new global tariff rate...,https://www.cbc.ca/news/politics/trump-global-...
4,41,2026-02-22-14-45-01 +0000,startribune,Armed man shot and killed after entering secur...,https://www.startribune.com/us-secret-service-...
87,36,2026-02-22-04-47-09 +0000,nypost,Iran president says Tehran won’t ‘bow’ as war ...,https://nypost.com/2026/02/21/world-news/iran-...
137,30,2026-02-21-21-18-52 +0000,nypost,Panicked Dems fear ‘zombie campaigns’ will se...,https://nypost.com/2026/02/21/us-news/californ...
5,28,2026-02-22-14-40-24 +0000,startribune,"Live: U.S., Canada tied 1-1 in men’s hockey go...",https://www.startribune.com/2026-winter-olympi...
116,28,2026-02-22-00-15-48 +0000,nypost,"Senior MP demands UK probe ex-Prince Andrew, P...",https://nypost.com/2026/02/21/world-news/senio...
30,26,2026-02-22-12-56-38 +0000,nypost,NYC area set for 2 feet of snow in first dange...,https://nypost.com/2026/02/22/us-news/nyc-area...
51,25,2026-02-22-11-00-00 +0000,startribune,"ICE detained a Minnesota teen, labeled him an ...",https://www.startribune.com/how-ice-labeled-a-...
33,25,2026-02-22-12-10-02 +0000,startribune,Homeland Security suspends TSA PreCheck and Gl...,https://www.startribune.com/homeland-security-...
32,24,2026-02-22-12-18-05 +0000,nyt,"Trump Looks Ahead to Summit With China’s Xi, b...",https://www.nytimes.com/2026/02/22/world/asia/...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
